In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ipdb

# Load Data

In [3]:
sl_data = pd.read_csv('./data/Smart Location Data Export.csv')

In [4]:
sl_data.head(2)

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,Com_VMT_Per_Worker,VMT_per_worker,VMT_tot_min,VMT_tot_max,VMT_tot_avg,GHG_per_worker,Annual_GHG,SLC_score,Shape_Length,Shape_Area
0,1,481130078254,481130078254,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,21.688738,27.496405,11.442995,82.636303,25.659327,24.499297,6369.817221,77.450956,3110.36082,297836.0831
1,2,481130078252,481130078252,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,21.379832,26.465754,11.442995,82.636303,25.659327,23.580987,6131.056669,78.898635,3519.46911,484945.1466


In [15]:
# completed by code "OD_batch.py"